<a href="https://colab.research.google.com/github/Stalin-alt/Summer-Analytics-2025/blob/main/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas geopy


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset.csv


In [ ]:

import pandas as pd
import numpy as np
from geopy.distance import geodesic

df = pd.read_csv("dataset.csv")

df['Occupancy'] = pd.to_numeric(df['Occupancy'], errors='coerce')
df['Capacity'] = pd.to_numeric(df['Capacity'], errors='coerce')
df['QueueLength'] = pd.to_numeric(df['QueueLength'], errors='coerce')
df['IsSpecialDay'] = pd.to_numeric(df['IsSpecialDay'], errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

traffic_map = {'low': 0, 'moderate': 0.5, 'high': 1}
df['Traffic'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0.5)

vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.3}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights).fillna(1.0)



In [ ]:

base_price = 10
lambda_ = 0.8
alpha, beta, gamma, delta = 0.5, 0.3, 0.2, 0.5

df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()
df['TrafficNorm'] = df['Traffic'] / 1

df['Demand'] = (
    alpha * df['OccupancyRate'] +
    beta * df['QueueNorm'] -
    gamma * df['TrafficNorm'] +
    delta * df['IsSpecialDay'] +
    df['VehicleWeight']
)



In [ ]:

df['DemandNorm'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())
df['Price'] = base_price * (1 + lambda_ * df['DemandNorm'])
df['Price'] = df['Price'].clip(lower=0.5 * base_price, upper=2 * base_price)

from scipy.spatial import KDTree

coords = list(zip(df['Latitude'], df['Longitude']))
tree = KDTree(coords)

neighbors = tree.query_ball_tree(tree, r=0.5 / 111)

final_prices = []
actions = []

for i, row in df.iterrows():
    price = row['Price']
    occ = row['Occupancy']
    cap = row['Capacity']
    nearby_idxs = [j for j in neighbors[i] if j != i]

    nearby_prices = df.iloc[nearby_idxs]['Price'].tolist()
    if not nearby_prices:
        final_prices.append(price)
        actions.append(None)
        continue

    min_price = min(nearby_prices)
    max_price = max(nearby_prices)

    if occ >= cap and price > min_price:
        final_prices.append(min_price - 1)
        actions.append('Suggest reroute')
    elif price < max_price:
        final_prices.append(price + 1)
        actions.append(None)
    else:
        final_prices.append(price)
        actions.append(None)



In [ ]:

df['FinalPrice'] = final_prices
df['Action'] = actions

print(" MODEL 3 (COMPETITION + BEHAVIOR AWARE) OUTPUT")
print(df[['ID', 'Occupancy', 'QueueLength', 'TrafficConditionNearby', 'VehicleType', 'Price', 'FinalPrice', 'Action']].head())



 MODEL 3 (COMPETITION + BEHAVIOR AWARE) OUTPUT
   ID  Occupancy  QueueLength TrafficConditionNearby VehicleType      Price  \
0   0         61            1                    low         car  12.061201   
1   1         64            1                    low         car  12.073359   
2   2         80            2                    low         car  12.231739   
3   3        107            2                    low         car  12.341161   
4   4        150            2                    low        bike  11.112384   

   FinalPrice Action  
0   13.061201   None  
1   13.073359   None  
2   13.231739   None  
3   13.341161   None  
4   12.112384   None  


In [ ]:
# MODEL 3 SUMMARY:

# Demand Function:
# Demand = α * OccupancyRate + β * QueueNorm - γ * TrafficNorm + δ * IsSpecialDay + VehicleWeight
# This is the same base demand model as Model 2, combining various real-world factors.

# Pricing:
# Initial price = base_price * (1 + λ * NormalizedDemand)
# This gives each lot a starting price based on demand only.

# Competitive Adjustment Logic:
# - Uses KDTree to quickly find nearby parking lots within 0.5 km.
# - If lot is full and price is higher than nearby ones → suggest reroute (lower price).
# - If price is too low compared to nearby lots → slightly raise price to stay competitive.

# Assumptions:
# - High demand features (occupancy, queues, special events) push price up.
# - Heavy traffic lowers demand and price.
# - Drivers may be influenced by price and location — competition matters.
# - Nearby cheaper lots should attract traffic, so we reroute or adjust price.

# Advantage over Model 1 & 2:
# Includes real-time competition between lots based on distance.
# Adapts price both from demand and nearby pricing context.
# Uses geospatial KDTree for efficient and fast nearby lot detection.



In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap

output_notebook()

source = ColumnDataSource(df)

p = figure(title="Model 3: Final Price vs Base Price", x_axis_label='Base Price (Demand-Based)', y_axis_label='Final Price (After Competition)')
p.circle(x='Price', y='FinalPrice', size=10, source=source,
         color=factor_cmap('Action', palette=['lightblue', 'red'], factors=list(df['Action'].fillna('None').unique())),
         legend_field='Action', alpha=0.6)
p.legend.title = 'Action'
show(p)
